### Monday, January 29, 2024

Trying this again ... 

conda activate langchain2

### Friday, January 26, 2024

Going to try to get this to run but instead of hitting OpenAI, use LMStudio.

I am skipping the HuggingFaceHub section and working in the OpenAI section.

### Wednesday, November 22, 2023

Running again, along with the corresponding video.

[Getting Started with GPT-3 vs. Open Source LLMs - LangChain #1](https://www.youtube.com/watch?v=nE2skSRWTTs&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F&index=1)

Start : OpenAI Usage => $1.63

End   : OpenAI Usage => $1.63


### Monday, November 20, 2023

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb


In [5]:
!ls /home/rob/Data2/huggingface/transformers/models--*

/home/rob/Data2/huggingface/transformers/models--bert-base-cased:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--bert-base-cased-finetuned-mrpc:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--bert-base-uncased:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--bert-large-uncased:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--camembert-base:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--castorini--tct_colbert-v2-hnp-msmarco:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--dbmdz--bert-large-cased-finetuned-conll03-english:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--distilbert-base-cased-distilled-squad:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--distilbert-base-uncased:
blobs  refs  snapshots

/home/rob/Data2/huggingface/transformers/models--distilbert-base-uncased-finetuned-ss

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

In [2]:
# !pip install -qU langchain

# Using LLMs in LangChain

LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## Hugging Face

We first need to install additional prerequisite libraries:

In [3]:
# !pip install -qU huggingface_hub

For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [8]:
import os
from getpass import getpass

os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass("HuggingFace API Key: ")

In [4]:
# import os
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'asdf'

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [11]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain


In [12]:
# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":1e-10}
)

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [13]:
# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)


In [14]:
question = "Which NFL team won the Super Bowl in the 2010 season?"

print(llm_chain.run(question))

# The above will time out at around ...
# 5m 33.9s

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ValueError: Error raised by inference API: Model google/flan-t5-xl time out

If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [15]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

# 5m 2.6s  ... This too will time out ...
# 5m 33.5s ... same time out period! Interesting ... !

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

It is a LLM, so we can try feeding in all questions at once:

In [16]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

# print(llm_chain.run(qs_str))

In [17]:
print(llm_chain.run(qs_str))

# 5m 33.7s ... time out!

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [9]:
# !pip install -qU openai

We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [28]:
import os

# os.environ['OPENAI_API_KEY'] = 'sk-asdf'

from getpass import getpass

# os.environ['OPENAI_API_KEY'] = getpass("OpenAI API Key: ")

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [29]:
# from langchain.llms import OpenAI

# davinci = OpenAI(model_name='text-davinci-003')

In [1]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

from langchain.llms import OpenAI

In [2]:
# Copied from some earlier cells ...

question = "Which NFL team won the Super Bowl in the 2010 season?"

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
# davinci = OpenAI(model_name='text-davinci-003')

davinci = OpenAI(base_url="http://localhost:1234/v1", api_key="NULL")

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Alternatively if using Azure OpenAI we do:

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="text-davinci-003"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [4]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

In [5]:
print(llm_chain.run(question))

# 0.8s

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The New England Patriots won Super Bowl XLVI in the 2010 season.


The same works again for multiple questions using `generate`:

In [6]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

# I was downloading "TheBloke/dolphin-2.5-mixtral-8x7b-GGUF" in LMStudio when I ran this ... 
# 55.9s 
# Nope! It's because you were using LMStudio and did NOT have all of the model loaded to the GPU
# but had some layers running on the CPU.
# I loaded all layers to the GPU and it took 8.6s! 
# Hmm and when I ran this a second time, it took 0.1s ... ?? ... restart the kernel, then try again, and it still took 0.1s!

# 1.0s 

LLMResult(generations=[[Generation(text=' ', generation_info={'finish_reason': 'stop', 'logprobs': None})], [], [], []], llm_output={'token_usage': {'total_tokens': 347, 'prompt_tokens': 346, 'completion_tokens': 1}, 'model_name': 'gpt-3.5-turbo-instruct'}, run=[RunInfo(run_id=UUID('77638956-348f-4d13-900b-e3f01fdc9141')), RunInfo(run_id=UUID('b14a3875-47bd-4464-9fce-160366854313')), RunInfo(run_id=UUID('f1a5c86d-d7d8-47d5-b820-fa0008bb82db')), RunInfo(run_id=UUID('877cbe34-6505-4e7f-93eb-afdf4af7eb01'))])

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [26]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))

The question "Which NFL team won the Super Bowl in the 2010 season?" can be answered by looking at the data from that year's Super Bowl. The winner of the Super Bowl is the team that scored more points than the other team.

The question "If I am 6 ft 4 inches, how tall am I in centimeters?" can be answered by converting the height from feet to meters and then from meters to centimeters.

The question "Who was the 12th person on the moon?" can be answered by looking at the list of astronauts who went to the moon. The 12th person on the moon is the one that went up with Apollo 9.

The question "How many eyes does a blade of grass have?" can be answered by counting the number of eyes on each blade of grass.

Therefore, the answer to these questions is:

1. The New York Giants won the Super Bowl in the 2010 season.
2. If I am 6 ft 4 inches, then my height in centimeters would be 193 cm.
3. The 12


Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `text-davinci-003` will be more likely to handle these more complex queries.

In [27]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

# 1.6s

1. The New York Giants won the Super Bowl in the 2010 season.
2. You are 6 ft 4 inches tall, which is equivalent to 193.2 cm or 193 centimeters.


---